<a href="https://colab.research.google.com/github/memelordmaddy/WiDS-2023/blob/main/Week4debugged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Resize
from torch.utils.data import DataLoader

In [7]:
from google.colab import drive
from torchvision.transforms import Resize, ToTensor, Normalize, RandomHorizontalFlip, RandomVerticalFlip, RandomRotation, ColorJitter
from torch.utils.data import random_split, Dataset

drive.mount('/content/gdrive')
yespath = '/content/gdrive/My Drive/yes'
nopath = '/content/gdrive/My Drive/no'

transform = transforms.Compose([
    Resize((240, 240)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

class CustomDataset(Dataset):
    def __init__(self, root, transform=None, label=0):
        self.root = root
        self.transform = transform
        self.label = label
        self.file_list = os.listdir(root)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root, self.file_list[idx])
        image = Image.open(img_name).convert('RGB')
        label = torch.tensor(self.label, dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, label

positive_dataset = CustomDataset(root=yespath, transform=transform, label=1)
negative_dataset = CustomDataset(root=nopath, transform=transform, label=0)

train_size_positive = int(0.8 * len(positive_dataset))
test_size_positive = len(positive_dataset) - train_size_positive

train_size_negative = int(0.8 * len(negative_dataset))
test_size_negative = len(negative_dataset) - train_size_negative

train_dataset = torch.utils.data.ConcatDataset([
    torch.utils.data.Subset(positive_dataset, range(train_size_positive)),
    torch.utils.data.Subset(negative_dataset, range(train_size_negative))
])

test_dataset = torch.utils.data.ConcatDataset([
    torch.utils.data.Subset(positive_dataset, range(train_size_positive, len(positive_dataset))),
    torch.utils.data.Subset(negative_dataset, range(train_size_negative, len(negative_dataset)))
])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
class BrainTumorCNN(nn.Module):
    def __init__(self):
        super(BrainTumorCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(115200, 256)
        self.fc2 = nn.Linear(256, 1)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxpool3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        logits = self.fc2(x)
        return logits

model = BrainTumorCNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



In [34]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0, checkpoint_path='checkpoint.pt'):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.checkpoint_path = checkpoint_path

    def __call__(self, val_loss, model):
        if self.best_score is None:
            self.best_score = val_loss
            self.save_checkpoint(val_loss, model)
        elif val_loss > self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = val_loss
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        torch.save(model.state_dict(), self.checkpoint_path)

early_stopping = EarlyStopping(patience=5, delta=0.0025)

In [35]:
from PIL import Image

epochs = 20

for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels.float().view(-1, 1)).item()

            predicted = torch.round(torch.sigmoid(outputs))
            total += labels.size(0)
            correct += (predicted == labels.view(-1, 1)).sum().item()

    val_loss /= len(test_loader)
    accuracy = correct / total * 100
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')

    early_stopping(val_loss, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break

model.load_state_dict(torch.load(early_stopping.checkpoint_path))

model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = torch.round(torch.sigmoid(outputs))
        test_total += labels.size(0)
        test_correct += (predicted == labels.view(-1, 1)).sum().item()

test_accuracy = test_correct / test_total * 100
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch [1/20], Loss: 0.1647, Val Loss: 1.0187, Accuracy: 80.39%
Epoch [2/20], Loss: 0.1191, Val Loss: 1.1947, Accuracy: 82.35%
Epoch [3/20], Loss: 0.3617, Val Loss: 1.1749, Accuracy: 66.67%
Epoch [4/20], Loss: 0.2493, Val Loss: 1.2073, Accuracy: 68.63%
Epoch [5/20], Loss: 0.1366, Val Loss: 0.6262, Accuracy: 76.47%
Epoch [6/20], Loss: 0.1112, Val Loss: 0.9620, Accuracy: 74.51%
Epoch [7/20], Loss: 0.5644, Val Loss: 0.9163, Accuracy: 80.39%
Epoch [8/20], Loss: 0.1148, Val Loss: 1.3607, Accuracy: 74.51%
Epoch [9/20], Loss: 0.3748, Val Loss: 0.8047, Accuracy: 80.39%
Epoch [10/20], Loss: 0.0687, Val Loss: 0.9597, Accuracy: 72.55%
Early stopping
Test Accuracy: 70.59%
